# 0.0 Imports

In [ ]:
import pandas as pd
import inflection
import math

## 0.1 Funções auxiliares

# 1.0 Carregando os dados

In [ ]:
df_sales_raw = pd.read_csv('data/train.csv', low_memory= False)
df_store_raw = pd.read_csv('data/store.csv', low_memory= False)


#merge
df_raw = pd.merge( df_sales_raw, df_store_raw, how='left', on='Store')

# 2.0 Descrição dos dados

In [ ]:
df1 = df_raw.copy()

## 2.1 Renomear colunas

In [ ]:
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo','StateHoliday',
            'SchoolHoliday', 'StoreType', 'Assortment','CompetitionDistance', 'CompetitionOpenSinceMonth',
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek','Promo2SinceYear', 'PromoInterval']

snakecase = lambda x: inflection.underscore(x)

cols_new = list( map(snakecase, cols_old) )

#renomear
df1.columns = cols_new

## 2.2 Dimensão dos dados

In [ ]:
print( 'Number of Rows:{}'.format( df1.shape[0]))
print( 'Number of Cols:{}'.format( df1.shape[1]))

## 2.3 Tipo de dados

In [ ]:
df1.dtypes

In [ ]:
df1['date'] = pd.to_datetime( df1['date'])
df1.dtypes

## 2.4 verificar os NA

In [ ]:
df1.isna().sum()

## 2.5 preencher os NA

In [ ]:
df1['competition_distance'].max()

In [ ]:
#competition_distance              2642
df1['competition_distance'] = df1['competition_distance'].apply( lambda x: 200000.0 if math.isnan ( x ) else x )

#competition_open_since_month    323348
df1['competition_open_since_month'] = df1.apply( lambda x: x['date'].month if math.isnan( x['competition_open_since_month']) else
                                                x['competition_open_since_month'], axis =1)

#competition_open_since_year     323348
df1['competition_open_since_year'] = df1.apply( lambda x: x['date'].month if math.isnan( x['competition_open_since_year']) else
                                                x['competition_open_since_year'], axis =1)
#promo2                               0
#promo2_since_week               508031
df1['promo2_since_week'] = df1.apply( lambda x: x['date'].month if math.isnan( x['promo2_since_week']) else
                                                x['promo2_since_week'], axis =1)
#promo2_since_year               508031
df1['promo2_since_year'] = df1.apply( lambda x: x['date'].month if math.isnan( x['promo2_since_year']) else
                                                x['promo2_since_year'], axis =1)

#promo_interval                  508031
month_map = {1:'Jan',2:'fev',3:'Mar',4:'Apr',5:'May',6:'Jun',7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}
#substituir NA por 0
df1['promo_interval'].fillna(0, inplace=True)
#transaformando data em mes
df1['month_map'] = df1['date'].dt.month.map(month_map)
#verificando se a loja fez parte da promocao
df1['is_promo'] = df1[['promo_interval','month_map']].apply( lambda x: 0 if x['promo_interval'] == 0 else 1 
                                                            if x['month_map'] in x['promo_interval'].split(',') else 0, axis=1)

In [ ]:
df1.isna().sum()

In [ ]:
df1.sample(10).T